In [1]:
import pandas as pd

otu_df = pd.read_csv('dataframes/OTU.txt', sep='\t')
hiera_blast = pd.read_csv('dataframes/hiera_BLAST.txt', sep='\t')
meta_map = pd.read_csv('dataframes/MCOSIS_map.txt', sep='\t')
meta = pd.read_csv('dataframes/antibiotics kazakh finished.csv', sep='\t')

In [2]:
meta = meta[['ID', 'sample', 'day of research', 'intervention', 'date of birth']]
meta_map_expanded = meta_map.set_index('#SampleID').fastqFile.str.split(',', expand=True).stack().reset_index(name='fastqFile').drop('level_1', axis=1)

# Соединение таблиц
merged = pd.merge(meta, meta_map_expanded, left_on='ID', right_on='fastqFile')

# Замена старых значений 'ID' на новые из '#SampleID'
meta['ID'] = merged['#SampleID']

# Показываем результат
meta

,ID,sample,day of research,intervention,date of birth
0,SMPL0,1,40,yes,23.10.2012
1,SMPL1,1,45,yes,23.10.2012
2,SMPL2,1,50,yes,23.10.2012
3,SMPL3,1,55,yes,23.10.2012
4,SMPL4,1,60,yes,23.10.2012
...,...,...,...,...,...
108,SMPL108,5,5,no,21.08.2014
109,SMPL109,5,15,no,21.08.2014
110,SMPL110,6,1,no,15.04.2014
111,SMPL111,6,5,no,15.04.2014


In [4]:
# Транспонирование otu_df
otu_df_transposed = otu_df.set_index('OTU').T
otu_df_transposed = otu_df_transposed.reset_index().rename(columns={'index': 'SampleID'})

# Переименование столбцов в meta_result для соответствия с otu_df_transposed
meta_result = meta.rename(columns={'ID': 'SampleID', 'intervention': 'GROUP'})

# Объединение таблиц
merged_df = pd.merge(meta_result, otu_df_transposed, on='SampleID')

# Переименование столбцов OTU
for col in merged_df.columns:
    if 'OTU' in col:
        merged_df = merged_df.rename(columns={col: col.replace('OTU_', 'OTU')})

# Вывод результата
merged_df_group = merged_df.drop(columns=['GROUP', 'date of birth'])
merged_df = merged_df.drop(columns=['GROUP', 'date of birth'])

merged_df

,SampleID,sample,day of research,OTU1,OTU2,OTU3,OTU4,OTU5,OTU6,OTU7,...,OTU4190,OTU4191,OTU4192,OTU4193,OTU4194,OTU4195,OTU4196,OTU4197,OTU4198,OTU4199
0,SMPL0,1,40,12,0,26,0,14,0,0,...,0,0,0,0,0,0,0,0,0,0
1,SMPL1,1,45,416,60,672,1,1641,59,0,...,0,0,0,0,0,0,0,0,0,0
2,SMPL2,1,50,101,8,7,0,567,1,0,...,0,0,0,0,0,0,0,0,0,0
3,SMPL3,1,55,3161,1505,11,0,3890,1,3,...,0,0,0,0,0,0,0,0,0,0
4,SMPL4,1,60,15,3,2,8,123,2,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,SMPL107,5,55,0,1,0,2,2,0,0,...,0,0,0,0,0,0,0,0,0,0
108,SMPL108,5,5,2452,1782,0,0,34,7,19,...,0,0,0,0,0,0,0,0,0,0
109,SMPL109,5,15,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
110,SMPL110,6,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
meta['SampleID'] = merged_df_group['SampleID'].astype(str)

# Удаление строк, где 'SampleID' равен 'nan'
meta = meta[meta['SampleID'] != 'nan']

# Теперь можно использовать dropna() для удаления оставшихся строк с NaN, если они есть
meta = meta.dropna(subset=['SampleID'])
meta['ID'] = meta.apply(lambda row: row['ID'] + '_before' if row['day of research'] <= 30 else row['ID'] + '_after', axis=1)
meta

,ID,sample,day of research,intervention,date of birth,SampleID
0,SMPL0_after,1,40,yes,23.10.2012,SMPL0
1,SMPL1_after,1,45,yes,23.10.2012,SMPL1
2,SMPL2_after,1,50,yes,23.10.2012,SMPL2
3,SMPL3_after,1,55,yes,23.10.2012,SMPL3
4,SMPL4_after,1,60,yes,23.10.2012,SMPL4
...,...,...,...,...,...,...
107,SMPL107_after,5,55,no,21.08.2014,SMPL107
108,SMPL108_before,5,5,no,21.08.2014,SMPL108
109,SMPL109_before,5,15,no,21.08.2014,SMPL109
110,SMPL110_before,6,1,no,15.04.2014,SMPL110
